In this notebook we tried to effective memory efficient way of encoding our target variable.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import classification_report

from joblib import dump
import os

In [2]:
import os
#import cv2
import pandas as pd
#import cupy as cp  # CuPy for GPU-based NumPy operations
import numpy as np
#import tensorflow as tf
import scipy
#from skimage.feature import local_binary_pattern
#from skimage.filters import gabor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Load the data,labels and groups**

In [3]:
# Memory-mapped file paths
output_dir = 'Extracted_features'
data_file = os.path.join(output_dir, 'data_file.dat')
labels_file = os.path.join(output_dir, 'labels_file.dat')
labels_encoded_file = os.path.join(output_dir,"labels_encoded.dat")
groups_file = os.path.join(output_dir, 'groups_file.dat')

# Known values
total_images = 17780  # Total original images + augmentations
feature_size = 49162   # Number of features per image 

# Load the memory-mapped arrays for reading
data = np.memmap(data_file, dtype='float32', mode='r', shape=(total_images, feature_size))
labels = np.memmap(labels_file, dtype='object', mode='r', shape=(total_images,))
labels_encoded = np.memmap(labels_encoded_file, dtype='int32', mode='w+', shape=(total_images,))
groups = np.memmap(groups_file, dtype='int32', mode='r', shape=(total_images,))

In [4]:
# Checking the shapes of data,labels and groups
print(data.shape)
print(labels.shape)
print(groups.shape)

(17780, 49162)
(17780,)
(17780,)


: 

49162 features and 17780 rows

**Label Encoding of target variable**

In [5]:
import numpy as np

# Step 1: Create a dictionary mapping unique labels to integers
unique_labels = np.unique(labels)
label_map = {label: idx for idx, label in enumerate(unique_labels)}

# Step 2: Process labels in chunks and encode using the dictionary
chunk_size = 1000

for start in range(0, len(labels), chunk_size):
    end = min(start + chunk_size, len(labels))
    
    label_chunk = labels[start:end].astype(str)  # Convert to string
    encoded_chunk = np.array([label_map[label] for label in label_chunk], dtype=np.int32)
    
    labels_encoded[start:end] = encoded_chunk
    labels_encoded.flush()

print("Label encoding completed using dictionary mapping.")
